# SD2 Text-to-Image 768x768

In [ ]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch

repo_id = "stabilityai/stable-diffusion-2"
pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, revision="fp16")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")



In [ ]:
prompt = "High quality photo of an astronaut riding a horse in space"
generator = torch.Generator("cuda").manual_seed(1)
image = pipe(
    prompt,
    guidance_scale=9,
    num_inference_steps=25,
    generator=generator,
).images[0]


In [ ]:
image

# SD2 Inpainting 512x512

In [ ]:
import PIL
import requests
import torch
from io import BytesIO

from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler


def download_image(url):
    response = requests.get(url)
    return PIL.Image.open(BytesIO(response.content)).convert("RGB")


img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
mask_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"

init_image = download_image(img_url).resize((512, 512))
mask_image = download_image(mask_url).resize((512, 512))

repo_id = "stabilityai/stable-diffusion-2-inpainting"
pipe = DiffusionPipeline.from_pretrained(
    repo_id,
    torch_dtype=torch.float16,
    revision="fp16",
)

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")



In [ ]:
init_image

In [ ]:
mask_image

In [ ]:
prompt = "Face of a yellow cat, high resolution, sitting on a park bench"
image = pipe(
    prompt=prompt,
    image=init_image,
    mask_image=mask_image,
    num_inference_steps=25
).images[0]


In [ ]:
image

# SD2 Super Resolution

In [ ]:
import requests
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionUpscalePipeline
import torch

# load model and scheduler
model_id = "stabilityai/stable-diffusion-x4-upscaler"
pipeline = StableDiffusionUpscalePipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipeline = pipeline.to("cuda")

# let's download an  image
url = "https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/sd2-upscale/low_res_cat.png"
response = requests.get(url)
low_res_img = Image.open(BytesIO(response.content)).convert("RGB")
low_res_img = low_res_img.resize((128, 128))


In [ ]:
low_res_img

In [ ]:
prompt = "a white cat"
upscaled_image = pipeline(prompt=prompt, image=low_res_img).images[0]


In [ ]:
upscaled_image

# SD2 Depth-to-Image

In [ ]:
import torch
import requests
from PIL import Image

from diffusers import StableDiffusionDepth2ImgPipeline

pipe = StableDiffusionDepth2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-depth",
    torch_dtype=torch.float16,
).to("cuda")


url = "http://images.cocodataset.org/val2017/000000039769.jpg"
init_image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
init_image

In [ ]:
prompt = "two tigers"
n_prompt = "bad, deformed, ugly, bad anotomy"
image = pipe(prompt=prompt, image=init_image, negative_prompt=n_prompt, strength=0.7).images[0]

In [ ]:
image

# SD Pix2Pix

In [ ]:
import PIL
import requests
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline

model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

url = "https://huggingface.co/datasets/diffusers/diffusers-images-docs/resolve/main/mountain.png"


def download_image(url):
    image = PIL.Image.open(requests.get(url, stream=True).raw)
    image = PIL.ImageOps.exif_transpose(image)
    image = image.convert("RGB")
    return image


image = download_image(url)



In [ ]:
image

In [ ]:
prompt = "snow covered peaks, snowy, rocky, lots of snow, dark"
generator = torch.Generator("cuda").manual_seed(5)
out_image = pipe(
    prompt,
    image=image,
    num_inference_steps=30,
    image_guidance_scale=1.3,
    guidance_scale=12,
    generator=generator,
).images[0]


In [ ]:
out_image

In [ ]:
# Morning:
# - try out the youtube demo
# - try depth2img
# - figure out what the parameters in img2img do
# - figure out how to do depth2img

# - See if --xformers helps webui perf